In [1]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import tensorflow as tf

In [2]:
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, BatchNormalization, Dropout, Flatten
from tensorflow.keras.models import Model, Sequential

In [3]:
model = Sequential([
    Conv2D(512, (3,3), input_shape=(48,48,3),activation = 'relu'),
    MaxPool2D(),
    BatchNormalization(),
    Conv2D(512,(3,3), activation = 'relu'),
    MaxPool2D(),
    BatchNormalization(),
    Conv2D(256,(3,3), activation = 'relu'),
    MaxPool2D(),
    BatchNormalization(),
    Conv2D(256,(3,3), activation = 'relu'),
    MaxPool2D(),
    BatchNormalization(),
    Flatten(),
    Dense(512,activation = 'relu'),
    Dropout(0.2),
    Dense(256,activation = 'relu'),
    Dense(7,activation = 'softmax')
])

In [4]:
model.load_weights('kdef_model.h5')

## Loading a test image and preprocessing it

In [5]:
test_image = cv2.imread('testimage_cropped.jpg')

In [6]:
cv2.imshow('display', test_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
test_image_resized = cv2.resize(test_image,(48,48))

In [8]:
cv2.imshow('display', test_image_resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Prediction!

In [9]:
def predict_emotion(img):
    img = cv2.resize(img, (48,48))
    prediction = model.predict(img.reshape(1,48,48,3))
    prediction = np.argmax(prediction, axis = -1)
    emotions = ['ANGRY','FEARFUL','DISGUSTED','HAPPY','SAD','SURPRISED','NEUTRAL']
    
    return emotions[int(prediction)]

In [10]:
predict_emotion(test_image_resized)

'SAD'

In [11]:
img = cv2.imread('./KDEF/AF09/AF09DIS.jpg')
img = cv2.resize(img, (48,48))

In [12]:
img.shape

(48, 48, 3)

In [13]:
cv2.imshow('Test', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
predict_emotion(img)

'SAD'

## Using HC to create a bounding box around face found in image

In [15]:
hc_path = 'C:\\Users\\Randy\\Venv\\metamo\Lib\\site-packages\\cv2\\data\\haarcascade_frontalface_default.xml'

In [16]:
hc = cv2.CascadeClassifier(hc_path)

In [17]:
faces = hc.detectMultiScale(test_image)

In [18]:
for face in faces:
    x, y, w, h = face
    cv2.rectangle(test_image,(x,y),(x+w, y+h),(0,0,255),3)

In [19]:
def draw_bounding_box(img):
    #hc_path = 'C:\\Users\\Randy\\Venv\\cv\Lib\\site-packages\\cv2\\data\\haarcascade_frontalface_default.xml'
    #hc = cv2.CascadeClassifier(hc_path)
    faces = hc.detectMultiScale(img)
    for face in faces:
        x, y, w, h = face
        cv2.rectangle(img,(x,y),(x+w, y+h),(0,0,255),3)
    return img

In [20]:
cv2.imshow("Faces Detected", test_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [21]:
hc_path = 'C:\\Users\\Randy\\Venv\\metamo\\Lib\\site-packages\\cv2\\data'

In [22]:
cv2.imshow("Faces Detected", draw_bounding_box(test_image))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
x, y, w, h = faces[0]
cropped_face = test_image[y:y+h, x:x+w]

In [24]:
def return_cropped_face(img):
    faces = hc.detectMultiScale(img)
    for (x,y,w,h) in faces:
        img = img[y:y+h, x:x+w]
    return img

In [25]:
cv2.imshow("Cropped Face", cropped_face)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [26]:
predict_emotion(cropped_face)

'SAD'

In [27]:
video_cap = cv2.VideoCapture(0)

In [28]:
if not video_cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    # Capture frame-by-frame
    ret, frame = video_cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # Our operations on the frame come here
    faces = hc.detectMultiScale(frame)
    
    for (x,y,w,h) in faces:
        face = frame[y:y+h, x:x+w]
        
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, 
            predict_emotion(face), 
            (50, 50), #top left
            font, 1, 
            (255, 0, 0), #BGR 
            2, 
            cv2.LINE_4)
    # Display the resulting frame
    cv2.imshow('frame', draw_bounding_box(frame))
    if cv2.waitKey(1) == ord('q'):
        break
# When everything done, release the capture
video_cap.release()
cv2.destroyAllWindows()